## Import dependancies and set environment determinism

In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''  # Hide GPU devices

os.environ['TF_CPP_MIN_LOG_LEVEL'] = '1'
import tensorflow as tf
#     tf.get_logger().setLevel('ERROR')
tf.compat.v1.logging.set_verbosity(
    0
)
import numpy as np
import random
import pandas as pd


SEED = 378
# SEED = 123
print(SEED)
def set_seeds(seed=SEED):
    os.environ['PYTHONHASHSEED'] = str(seed)
    random.seed(seed)
    tf.random.set_seed(seed)
    np.random.seed(seed)

def set_global_determinism(seed=SEED):
    set_seeds(seed=seed)

    os.environ['TF_DETERMINISTIC_OPS'] = '1'
    os.environ['TF_CUDNN_DETERMINISTIC'] = '1'

    tf.config.threading.set_inter_op_parallelism_threads(1)
    tf.config.threading.set_intra_op_parallelism_threads(1)

# Call the above function with seed value
set_global_determinism(seed=SEED)

2024-04-22 10:59:44.760178: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory


378


In [1]:
import glacierml as gl
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers.experimental import preprocessing
from tqdm import tqdm

2024-04-22 13:50:47.434724: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-04-22 13:50:47.434749: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


In [3]:
def run_model(model_path, n,l1,l2,loss):
            
    normalizer = preprocessing.Normalization(axis=-1)
    normalizer.adapt(np.array(trfeat[n]))

    model = gl.build_dnn_model(
        normalizer, learning_rate = 0.01, 
        layer_1 = l1, layer_2 = l2,loss = loss
    )
    model_history = model.fit(
        trfeat[n],
        trlabs[n],
        validation_split=0.2,
        callbacks = [callback],
        verbose=0, 
        epochs=500
    )
    model_filename = os.path.join(model_path)
    model.save(model_filename)
    return model

In [4]:
callback = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',
    min_delta = 0.001,
    patience = 10,
    verbose = 0,
    mode = 'auto',
    baseline = None,
    restore_best_weights = True
)

In [3]:
home_path = '/home/simonhans/glacierml'
[
        data_path, RGI_path, glathida_path, 
        coregistration_testing_path, 
        arch_test_path, LOO_path
] = gl.set_paths(home_path)

In [4]:
list(gl.coregister_data(data_path,'4'))

['RGIId',
 'RGI Centroid Distance',
 'SURVEY_DATE',
 'AREA',
 'MEAN_THICKNESS_UNCERTAINTY',
 'GLACIER_NAME',
 'Thickness',
 'GLIMSId',
 'BgnDate',
 'EndDate',
 'CenLon',
 'CenLat',
 'O1Region',
 'O2Region',
 'Area',
 'Zmin',
 'Zmax',
 'Zmed',
 'Slope',
 'Aspect',
 'Lmax',
 'Status',
 'Connect',
 'Form',
 'TermType',
 'Surging',
 'Linkages',
 'Name',
 'region',
 'AVG Radius',
 'distance test',
 'perc difference']

In [6]:
gl.load_RGI(RGI_path)

,RGIId,GLIMSId,BgnDate,EndDate,CenLon,CenLat,O1Region,O2Region,Area,Zmin,...,Aspect,Lmax,Status,Connect,Form,TermType,Surging,Linkages,Name,region
0,RGI60-01.00001,G213177E63689N,20090703,-9999999,-146.8230,63.6890,1,2,0.360,1936,...,346,839,0,0,0,0,9,9,NaN,01
1,RGI60-01.00002,G213332E63404N,20090703,-9999999,-146.6680,63.4040,1,2,0.558,1713,...,162,1197,0,0,0,0,9,9,NaN,01
2,RGI60-01.00003,G213920E63376N,20090703,-9999999,-146.0800,63.3760,1,2,1.685,1609,...,175,2106,0,0,0,0,9,9,NaN,01
3,RGI60-01.00004,G213880E63381N,20090703,-9999999,-146.1200,63.3810,1,2,3.681,1273,...,195,4175,0,0,0,0,9,9,NaN,01
4,RGI60-01.00005,G212943E63551N,20090703,-9999999,-147.0570,63.5510,1,2,2.573,1494,...,181,2981,0,0,0,0,9,9,NaN,01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
216496,RGI60-19.02748,G322268E53986S,20020502,-9999999,-37.7325,-53.9860,19,3,0.042,310,...,315,255,0,0,0,0,9,9,NaN,19
216497,RGI60-19.02749,G323864E54831S,20030207,-9999999,-36.1361,-54.8310,19,3,0.567,330,...,200,1130,0,0,0,0,9,9,NaN,19
216498,RGI60-19.02750,G322698E54188S,20030207,-9999999,-37.3018,-54.1884,19,3,4.118,10,...,308,4329,0,0,0,1,9,9,NaN,19
216499,RGI60-19.02751,G269573E68866S,19870101,-9999999,-90.4266,-68.8656,19,1,0.011,170,...,122,106,0,0,0,1,9,9,AQ6C10200013,19


In [ ]:
['RGIId','Area','Zmin','Zmax','Zmed','Slope','Aspect','Lmax']

In [6]:
df = gl.coregister_data(data_path,'4')
df = df[[
    'RGIId','CenLon','CenLat','Area','Zmin','Zmed','Zmax',
    'Slope','Aspect','Lmax','Thickness'
]]
df = df.sample(frac = 1,random_state = 0)
df = df.reset_index().drop('index', axis = 1)

dft = df.copy()

trfeat = {}
trlabs = {}
tefeat = {}
telabs = {}

for n in df.index:
    mask = df.index.isin([n])
    trfeat[n] = df.loc[~mask].drop(['RGIId','Thickness'], axis = 1)
    tefeat[n] = df.loc[mask].drop(['RGIId','Thickness'], axis = 1)
    trlabs[n] = df['Thickness'].loc[~mask]
    telabs[n] = df['Thickness'].loc[mask]

model = {}
model_history = {}
normalizer = {}

for n in tqdm(df.index):
    model_path = os.path.join(LOO_path, str(n))
    if os.path.isdir(model_path) == True:
#         continue
        model[n] = gl.load_dnn_model(model_path)
    elif os.path.isdir(model_path) == False:
        set_global_determinism(seed=SEED)
        model[n] = run_model(model_path, n,l1 = 6,l2 = 2,loss = 'mae')
rgi_est_pth = os.path.join(LOO_path, 'rgi_est_raw.pkl')

RGI = gl.load_RGI(RGI_path)
rfp = RGI[list(df)[:-1]]

for n in tqdm(model.keys()):
    preds = pd.Series(
        model[n].predict(rfp.drop('RGIId',axis = 1)).flatten(), name = n
    )
    RGI = pd.concat([RGI,preds], axis = 1)
RGI.to_pickle(rgi_est_pth)



  0%|                                                   | 0/273 [00:00<?, ?it/s]2024-04-22 10:59:47.819600: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-04-22 10:59:47.819618: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
100%|█████████████████████████████████████████| 273/273 [17:15<00:00,  3.79s/it]


In [7]:
model_path

'/home/simonhans/glacierml/models/LOO/272'

In [8]:
LOO_path

'/home/simonhans/glacierml/models/LOO'